# [AlexNet]

*KU LeeDongGyu*

### Contents

1. Almost Original Alexnet
2. My Alexnet
```
1) Size = 64,
2) Size = 48,
```
3. Size = 48, No Early Stopping
```
1) Epoch = 50
2) Epoch = 100
```

### Install Packages

### Module

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
cd /content/drive/My Drive/Colab Notebooks/project

/content/drive/My Drive/Colab Notebooks/project


In [0]:
import sys

In [0]:
# 모듈로 받을 경로 확인
sys.path

['',
 '/env/python',
 '/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/root/.ipython']

In [0]:
# 내 노트북이 아닌, 전산실 컴퓨터의 colab에서 돌렸으므로, 다시돌리려면 경로 수정할것!
sys.path.append("/content/drive/My Drive/Colab Notebooks/project")

In [0]:
from lrn import LRN #만든 모듈, class
from f1score import macro_f1score, weighted_f1score
from pool_helper import PoolHelper

In [0]:
import pandas as pd
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten , ZeroPadding2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D 
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

In [0]:
os.getcwd()

'/content/drive/My Drive/Colab Notebooks/project'

## 1. Almost Original Alexnet

In [0]:
# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48,3]) 
x_valid = np.array(x_valid).reshape([-1,48,48,3]) 
x_test = np.array(x_test).reshape([-1,48,48,3]) 

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
# uint는 부호없는 정수로, 타입을 바꿔줘야함!
size=227

In [0]:
x_train.shape

(28698, 48, 48, 3)

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,3],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_train_zoom.shape

(28698, 227, 227, 3)

In [0]:
x_train = x_train / 255 # x_train, x_train_zoom 모두 0과 1사이에 값이 위치하게 설정.

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,3])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,3],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_valid_zoom.shape

(3589, 227, 227, 3)

In [0]:
 x_valid = x_valid / 255

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,3])
x_test_zoom = np.zeros([x_test.shape[0],size,size,3],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) / 255

In [0]:
x_test_zoom.shape

(3588, 227, 227, 3)

In [0]:
x_test = x_test / 255

In [0]:
# Alexnet을 최대한 논문에 가깝게 맞춰 모형작성.
# 또한, Data Augmentation은 컴퓨터 성능의 한계로 하지 않음.
# 출처 : https://github.com/eweill/keras-deepcv/blob/master/models/classification/alexnet.py

def Alexnet(img_shape=(227, 227, 3), n_classes=10, l2_reg=0.,weights=None):

    # Initialize model
    alexnet = Sequential()

    # Layer 1
    alexnet.add(Conv2D(96, (11, 11), input_shape=img_shape, strides=4,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer1_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 2
    alexnet.add(Conv2D(256, (5, 5), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer2_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))
    
    # Layer 3
    alexnet.add(Conv2D(384, (3, 3), padding='same', strides=1,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer3_LRN'))
    
    # Layer 4
    alexnet.add(Conv2D(384, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer4_LRN'))

    # Layer 5
    alexnet.add(Conv2D(256, (3, 3), padding='same'))
    alexnet.add(Activation('relu'))
    alexnet.add(LRN(name='layer5_LRN'))
    alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

    # Layer 6
    alexnet.add(Flatten())
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 7
    alexnet.add(Dense(4096,kernel_regularizer=l2(l2_reg)))
    alexnet.add(Activation('relu'))
    alexnet.add(Dropout(0.5))

    # Layer 8
    alexnet.add(Dense(n_classes))
    alexnet.add(Activation('softmax'))

    if weights is not None:
        alexnet.load_weights(weights)

    return alexnet

In [0]:
model = Alexnet(img_shape=(227, 227, 3), n_classes=7, l2_reg=0.,weights=None)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
#모수가 데이터에 비해 굉장히 많지만, 일단 진행
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 55, 55, 96)        34944     
_________________________________________________________________
activation (Activation)      (None, 55, 55, 96)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_1 (Activation)    (None, 27, 27, 256)       0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 27, 27, 256)       0

Too many parameters compared to the data. 

## 2. My Alexnet

In [0]:
# Alexnet을 개조
# Data Augmentation은 컴퓨터 성능의 한계로 못하기 때문에 변형함.

# 주의 !!!!기본은 모형을 조절하는 것이 아닌, 데이터를 뻥튀기 하는 것임을 늘 잊지말기!!!!

# data import
x_train = pd.read_csv("mydata/X_train.csv",header=0,index_col=0)
x_valid = pd.read_csv("mydata/X_private_test.csv",header=0,index_col=0)
x_test = pd.read_csv("mydata/X_public_test.csv",header=0,index_col=0)
y_train = pd.read_csv("mydata/y_train.csv",header=0,index_col=0)
y_valid = pd.read_csv("mydata/y_private_test.csv",header=0,index_col=0)
y_test = pd.read_csv("mydata/y_public_test.csv",header=0,index_col=0)

In [0]:
# data handling
x_train = np.array(x_train).reshape([-1,48,48,3]) 
x_valid = np.array(x_valid).reshape([-1,48,48,3]) 
x_test = np.array(x_test).reshape([-1,48,48,3]) 

y_train=to_categorical(y_train) # one hot encoding
y_valid=to_categorical(y_valid)
y_test=to_categorical(y_test)

In [0]:
# data handling
size = 64 #적당한 크기로 잡음.
x_train = np.array(x_train).reshape([-1,48,48,3])

In [0]:
x_train_zoom = np.zeros([x_train.shape[0],size,size,3],dtype="float32")

for i in range(x_train.shape[0]):
    x_train_zoom[i,:] = cv2.resize(x_train[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_train = x_train / 255 

In [0]:
x_valid = np.array(x_valid).reshape([-1,48,48,3])
x_valid_zoom = np.zeros([x_valid.shape[0],size,size,3],dtype="float32")
for i in range(x_valid.shape[0]):
    x_valid_zoom[i,:] = cv2.resize(x_valid[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
 x_valid = x_valid / 255

In [0]:
x_test = np.array(x_test).reshape([-1,48,48,3])
x_test_zoom = np.zeros([x_test.shape[0],size,size,3],dtype="float32")
for i in range(x_test.shape[0]):
    x_test_zoom[i,:] = cv2.resize(x_test[i,:].astype('uint8'), (size, size),
                                  interpolation=cv2.INTER_CUBIC).reshape(size,size,3) /255

In [0]:
x_test = x_test / 255

#### 1) Size = 64

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 64x64x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 14 x 14 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 14 x 14 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 12 x 12 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 12 x 12 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(64, 64, 3), n_classes=7, l2_reg=0.,
	weights=None):

	# Initialize model
	alexnet = Sequential()

	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(64, 64, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_30 (Conv2D)           (None, 30, 30, 24)        1824      
_________________________________________________________________
activation_48 (Activation)   (None, 30, 30, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 30, 30, 24)        0         
_________________________________________________________________
max_pooling2d_12 (MaxPooling (None, 14, 14, 24)        0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 14, 14, 64)        13888     
_________________________________________________________________
activation_49 (Activation)   (None, 14, 14, 64)        0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 14, 14, 64)       

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
# early stopping 
model.fit(x_train_zoom,y_train,batch_size=128, validation_data=(x_valid_zoom,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 6s 219us/sample - loss: 1.8263 - acc: 0.2466 - macro_f1score: 3.2560e-05 - weighted_f1score: 8.1400e-06 - val_loss: 1.8151 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 5s 173us/sample - loss: 1.7962 - acc: 0.2521 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.7344 - val_acc: 0.2789 - val_macro_f1score: 0.0013 - val_weighted_f1score: 1.4053e-04
Epoch 3/100
28698/28698 [==============================] - 5s 173us/sample - loss: 1.6699 - acc: 0.3263 - macro_f1score: 0.0749 - weighted_f1score: 0.0127 - val_loss: 1.5505 - val_acc: 0.3948 - val_macro_f1score: 0.1319 - val_weighted_f1score: 0.0263
Epoch 4/100
28698/28698 [==============================] - 5s 173us/sample - loss: 1.5210 - acc: 0.4088 - macro_f1score: 0.1546 - weighted_f1score: 0.0290 - val_loss: 1.4

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test_zoom,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 80us/sample - loss: 1.6005 - acc: 0.5543 - macro_f1score: 0.4691 - weighted_f1score: 0.0748

Accuracy: 0.5543, Macro F1 Score: 0.4691, Weighted F1 Score: 0.0748


#### 2) Size = 48

In [0]:
# 다음의 절차로 모형을 개조한다.

# 1. 227의 대략 1/4 연산인 64로 이미지사이즈를 재조정한다.
# 2. 모수와 관련이 가장 깊은 fc층에서, 기존의 4096개의 노드를 1/16 (비율) 배 만큼, 즉 256개로줄인다.
# 3. 다음과 같이 모형을 재구성한다.

# convolution layer
# 입력 : 64x64x1
# 첫번째 층 : 5x5 필터 24장, strides = 2 -> maxpooling 3x3 , stirdes = 2   ===> 10 x 10 x 24 feature map
# 두번째 층 : 3x3 필터 64장, strides = 1, padding = "same"                 ===> 14 x 14 x 64 feature map
# 세번째 층 : 3x3 필터 96장, strides = 1                                   ===> 12 x 12 x 96 feature map
# 네번째 층 : 3x3 필터 96장, strides = 1, padding = "same"                 ===> 12 x 12 x 96 feature map
# 다섯째 층 : 3x3 필터 64장, strides = 1 -> maxpooling 3x3 , strides = 2   ===> 4 x 4 x 64 feature map

# fc layer
# 여섯째 층 : 노드 256개, dropout = 0.5
# 일곱째 층 : 노드 256개, dropout = 0.5
# 여덟째 층 : 노드 7개



def Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0., weights=None):

	# Initialize model
	alexnet = Sequential()


	# Layer 1
	alexnet.add(Conv2D(24, (5, 5), input_shape=img_shape, strides=2,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer1_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 2
	alexnet.add(Conv2D(64, (3, 3), padding='same',strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer2_LRN'))

	# Layer 3
	alexnet.add(Conv2D(96, (3, 3),  strides=1,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer3_LRN'))

	# Layer 4
	alexnet.add(Conv2D(96, (3, 3), padding='same'))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer4_LRN'))

	# Layer 5
	alexnet.add(Conv2D(64, (3, 3)))
	alexnet.add(Activation('relu'))
	alexnet.add(LRN(name='layer5_LRN'))
	alexnet.add(MaxPooling2D(pool_size=(3, 3),strides=2))

	# Layer 6
	alexnet.add(Flatten())
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))

	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 7
	alexnet.add(Dense(256,kernel_regularizer=l2(l2_reg)))
	alexnet.add(Activation('relu'))
	alexnet.add(Dropout(0.5))

	# Layer 8
	alexnet.add(Dense(n_classes))
	alexnet.add(Activation('softmax'))

	if weights is not None:
		alexnet.load_weights(weights)

	return alexnet

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_10 (Conv2D)           (None, 22, 22, 24)        1824      
_________________________________________________________________
activation_16 (Activation)   (None, 22, 22, 24)        0         
_________________________________________________________________
layer1_LRN (LRN)             (None, 22, 22, 24)        0         
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 10, 10, 24)        0         
_________________________________________________________________
conv2d_11 (Conv2D)           (None, 10, 10, 64)        13888     
_________________________________________________________________
activation_17 (Activation)   (None, 10, 10, 64)        0         
_________________________________________________________________
layer2_LRN (LRN)             (None, 10, 10, 64)       

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
early_stopping = EarlyStopping(monitor='val_macro_f1score',patience=3,verbose=1,mode='max')

In [0]:
# early stopping 
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100,callbacks=[early_stopping])

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 8s 276us/sample - loss: 1.8287 - acc: 0.2425 - macro_f1score: 1.3451e-04 - weighted_f1score: 3.5479e-05 - val_loss: 1.8277 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 7s 238us/sample - loss: 1.7483 - acc: 0.2883 - macro_f1score: 0.0371 - weighted_f1score: 0.0080 - val_loss: 1.6347 - val_acc: 0.3536 - val_macro_f1score: 0.0733 - val_weighted_f1score: 0.0159
Epoch 3/100
28698/28698 [==============================] - 7s 238us/sample - loss: 1.5577 - acc: 0.3871 - macro_f1score: 0.1368 - weighted_f1score: 0.0268 - val_loss: 1.5063 - val_acc: 0.4207 - val_macro_f1score: 0.1426 - val_weighted_f1score: 0.0281
Epoch 4/100
28698/28698 [==============================] - 7s 239us/sample - loss: 1.4693 - acc: 0.4320 - macro_f1score: 0.1724 - weighted_f1score: 0.0326 - val_loss: 1.4212 - val_ac

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 100us/sample - loss: 2.1548 - acc: 0.5326 - macro_f1score: 0.4578 - weighted_f1score: 0.0739

Accuracy: 0.5326, Macro F1 Score: 0.4578, Weighted F1 Score: 0.0739


## 3. Size = 48 , No Early Stopping
---

### 1) Epoch = 50

In [0]:
### early stopping 없이 ###

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
# early stopping 
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=50)

Train on 28698 samples, validate on 3589 samples
Epoch 1/50
28698/28698 [==============================] - 8s 271us/sample - loss: 1.8248 - acc: 0.2453 - macro_f1score: 4.3788e-05 - weighted_f1score: 9.5785e-06 - val_loss: 1.8170 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/50
28698/28698 [==============================] - 7s 236us/sample - loss: 1.8097 - acc: 0.2511 - macro_f1score: 4.5351e-05 - weighted_f1score: 9.5663e-06 - val_loss: 1.8015 - val_acc: 0.2449 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 3/50
28698/28698 [==============================] - 7s 239us/sample - loss: 1.7502 - acc: 0.2763 - macro_f1score: 0.0178 - weighted_f1score: 0.0021 - val_loss: 1.7131 - val_acc: 0.3040 - val_macro_f1score: 0.0314 - val_weighted_f1score: 0.0036
Epoch 4/50
28698/28698 [==============================] - 7s 237us/sample - loss: 1.6687 - acc: 0.3261 - macro_f1score: 0.0765 - weighted_f1score: 0.0122 - val_loss: 1.5

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 93us/sample - loss: 3.4816 - acc: 0.4822 - macro_f1score: 0.4418 - weighted_f1score: 0.0698

Accuracy: 0.4822, Macro F1 Score: 0.4418, Weighted F1 Score: 0.0698


### 2) Epoch = 100

In [0]:
#내 데이터 맞춤형 모형
model = Alexnet(img_shape=(48, 48, 3), n_classes=7, l2_reg=0.,weights=None)

In [0]:
# 여기는 학습의 효율을 위해. Adam으로 넘어간다.
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001), loss='categorical_crossentropy',
              metrics=['accuracy',macro_f1score,weighted_f1score])

In [0]:
# early stopping 
model.fit(x_train,y_train,batch_size=128, validation_data=(x_valid,y_valid) , epochs=100)

Train on 28698 samples, validate on 3589 samples
Epoch 1/100
28698/28698 [==============================] - 8s 277us/sample - loss: 1.8245 - acc: 0.2488 - macro_f1score: 0.0000e+00 - weighted_f1score: 0.0000e+00 - val_loss: 1.8103 - val_acc: 0.2441 - val_macro_f1score: 0.0000e+00 - val_weighted_f1score: 0.0000e+00
Epoch 2/100
28698/28698 [==============================] - 7s 240us/sample - loss: 1.7807 - acc: 0.2639 - macro_f1score: 0.0066 - weighted_f1score: 9.0251e-04 - val_loss: 1.6865 - val_acc: 0.3129 - val_macro_f1score: 0.0581 - val_weighted_f1score: 0.0070
Epoch 3/100
28698/28698 [==============================] - 7s 238us/sample - loss: 1.6090 - acc: 0.3611 - macro_f1score: 0.1149 - weighted_f1score: 0.0217 - val_loss: 1.5184 - val_acc: 0.4207 - val_macro_f1score: 0.1479 - val_weighted_f1score: 0.0285
Epoch 4/100
28698/28698 [==============================] - 7s 240us/sample - loss: 1.4974 - acc: 0.4157 - macro_f1score: 0.1641 - weighted_f1score: 0.0307 - val_loss: 1.4509 - va

In [0]:
_, acc, mac_f1, wei_f1 = model.evaluate(x_test,y_test,batch_size=128)
print("\nAccuracy: {:.4f}, Macro F1 Score: {:.4f}, Weighted F1 Score: {:.4f}".format(acc,mac_f1,wei_f1))

3588/3588 [==============================] - 0s 95us/sample - loss: 4.2640 - acc: 0.5307 - macro_f1score: 0.4857 - weighted_f1score: 0.0770

Accuracy: 0.5307, Macro F1 Score: 0.4857, Weighted F1 Score: 0.0770
